In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd "/content/drive/My Drive/EVA4/Notebooks/S10"

/content/drive/My Drive/EVA4/Notebooks/S10


In [3]:
!pip install albumentations

In [4]:
from vision import dataloaders
from vision.utils import Helper
from vision.models.resnet import ResNet18
from vision.imagetransforms import TorchTransforms,  album_transforms
import torch
from torchvision import  transforms
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from vision.train_test_model import RunModel
from vision.lr_finder import LRFinder
from torch.optim.lr_scheduler import ReduceLROnPlateau
from vision import plot_grad
from vision import train_test_model

  pip install torch-lr-finder -v --global-option="amp"


In [0]:
helper = Helper()

In [6]:
mean, std = helper.get_mean_and_std('cifar10')
# mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

Files already downloaded and verified


In [0]:
# mean = np.array([0.4914, 0.4822, 0.4465])
# std = np.array([0.2023, 0.1994, 0.2010])

In [8]:
mean, std

(array([0.49139968, 0.48215841, 0.44653091]),
 array([0.24703223, 0.24348513, 0.26158784]))

In [0]:
test_transforms = [ transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                  ]

In [0]:
torch_transforms = TorchTransforms(test_transforms)

In [11]:
aub = album_transforms()

Files already downloaded and verified


In [0]:
train_transforms = aub
test_transforms = torch_transforms.testTransform()
loader = dataloaders.Loader(train_transforms, test_transforms, 'cifar10')

In [13]:
train_dataloader  = loader.dataloader_train()
test_dataloader = loader.dataloader_test()
gradcam_dataloader = loader.dataloader_gradcam()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
Net = ResNet18()

In [0]:
# size =  (3, 32, 32)
# helper.model_summary(Net, size)

In [0]:
# L2 = 1e-4
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(Net.parameters(), lr=1e-8,momentum=0.9, weight_decay=L2)#, nesterov=True)
# lr_finder = LRFinder(Net, optimizer, criterion, device="cuda")
# lr_finder.range_test(train_dataloader, val_loader=test_dataloader, end_lr=1, num_iter=800, step_mode="exp")
# lr_finder.plot() 
# lr_finder.reset()

In [0]:
## Best_lr = lr_finder.history['lr'][lr_finder.history['loss'].index(lr_finder.best_loss)]

In [0]:
# loss = lr_finder.history['loss']
# lr = lr_finder.history['lr']
# Best_lr=lr[loss.index(min(loss))]


In [0]:
Best_lr = 0.000248313310529557

In [19]:
Best_lr

0.000248313310529557

In [0]:
# 0.000248313310529557 new
# 0.0018127308811846986
#  0.0039810717055349725
# 0.0002772407996741775
# 0.003228004742743393

In [0]:
epochs = 50
# L1 = 0.001
L1 = 0
L2 = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(Net.parameters(), lr=Best_lr,momentum=0.9, weight_decay=L2)#, nesterov=True)
# scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience = 2,min_lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

In [0]:
# run = RunModel(Net, train_dataloader, test_dataloader, optimizer, scheduler, epochs, L1, criterion=criterion)
# run.train_test()

Modified

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
test_acc = []
train_acc = []
trainloss_ = []
testloss_ = []
losses = []
accuracies = []

epochs = 50
for epoch in range(1, epochs + 1):
    for param_group in optimizer.param_groups:
      print("lr= ",param_group['lr'])
    print(f'Epoch {epoch}:')
    trainacc, loss = train_test_model.train(Net, train_dataloader, device, optimizer, criterion)
    train_acc.append(trainacc)
    testacc, loss1 = train_test_model.val(Net, test_dataloader, device, criterion, losses, accuracies)
    test_acc.append(testacc)
    trainloss_.append(loss)
    testloss_.append(loss1)
    scheduler.step(loss1)
print('Finished Training')

lr=  0.000248313310529557
Epoch 1:
---------


  0%|          | 0/1563 [00:00<?, ?it/s]

Testing: Average loss: 0.0390, Accuracy: 5458/10000 (54.58%)

lr=  0.000248313310529557
Epoch 2:
---------


  0%|          | 0/1563 [00:00<?, ?it/s]

Testing: Average loss: 0.0314, Accuracy: 6433/10000 (64.33%)

lr=  0.000248313310529557
Epoch 3:
---------


...

  0%|          | 0/1563 [00:00<?, ?it/s]

Testing: Average loss: 0.0179, Accuracy: 8455/10000 (84.55%)

lr=  7.759790954048656e-06
Epoch 34:
---------


  0%|          | 0/1563 [00:00<?, ?it/s]

Testing: Average loss: 0.0178, Accuracy: 8462/10000 (84.62%)

lr=  7.759790954048656e-06
Epoch 35:
---------


Loss=0.04 Accuracy=99.08:  81%|████████▏ | 1270/1563 [00:36<00:08, 35.06it/s]

In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
import warnings
warnings.simplefilter('ignore')
it = iter(gradcam_dataloader)
fig=plt.figure(figsize=(14, 16))
count = 0
while(count != 2):
    images, labels = it.next()
    output = Net(images.cuda())
    pred = output.argmax(dim=1, keepdim=True)
    if(classes[pred] != classes[labels]):
      plot_grad.get_gradcam(Net, images, classes, str(classes[labels]), str(classes[pred]))
      count=count+1

In [0]:
helper.plot_acc_graph(train_acc, test_acc, epochs)

In [0]:
helper.plot_loss_graph(trainloss_, testloss_, epochs)